In [1]:
import os
import re
import pandas as pd
import json

In [2]:
# function to process the extracted financial value (if extracted)
def process_value(value):
    if(value[0]=="{" or value[-1]=="}"):
        return value
    value = value.replace(",", "")
    if(len(value)!=0):
        if(value[0] == "("): 
            value = value.replace("(", "-")
        if(value[-1] == ")"):
            value = value.replace(")", "")
    return value

In [12]:
def extract_financial_data(file_path):
    
#EXTRACTING LINES FROM THE TEXT FILE.

    line_array=[]
    f = open(file_path, "r")
    for line in f.readlines():
        line_array.append((" ").join(line.split()))

#FIND THE STARTING AND ENDING INDEX OF FINANCIAL DATA AND EXTRACTING THE LINES CONTAINING FINANCIAL DATA..
    
    # most of the files have financial data confined between two lines having "dd Month YYYY" string.
    bounding_regex = re.compile(r'(\d(\d)?\s+)(\w+)\s+\d\d\d\d$') 
    found=False
    bounding_string=''
    for line in line_array:
        match = bounding_regex.search(line)
        if match and found == False:
            bounding_string=match.group()
            found=True

    financial_data=line_array
    if(bounding_string!=''):
        bounding_index=[line_array.index(l) for l in line_array if bounding_string in l] # index of lines containing the bounding string.
        # if bounding string is present at the starting index 
        if(len(bounding_index)==1):
            financial_data=financial_data[bounding_index[0]+1:]
        # if we are able to find the starting and ending index using bounding string.
        if(len(bounding_index)>1):
            financial_data = financial_data[bounding_index[0]+1:bounding_index[1]]
        for line in financial_data:
            # lines having "STATEMENTS" or "INFORMATION" or "NOTES TO THE ACCOUNTS" can act as the ending index for the financial data.
            if("statement" in line.lower() or "information" in line.lower() or "notes to the accounts" in line.lower()):
                financial_data = financial_data[:financial_data.index(line)]
                break
    
    # (optional-does not make any change to the output-just reduces the redundant data-specific to the problem) 
    if(len(financial_data)>15):
        financial_data=financial_data[:15]
    
# CODE TO FIND THE LINE - NOTES|NOTE (OPTIONAL) YEAR1 (OPTIONAL) YEAR2 to find the index of "2019" if it is there in the file

    regex = re.compile(r'((Note|Notes)\s)?((\d){4}\s)?(\d){4}$')
    # str_arr will be of the format [curr_year, prev_year] or [curr_year]
    str_arr=[]
    for line in financial_data:
        match = regex.search(line)
        if match:
            financial_data=financial_data[financial_data.index(line)+2:]
            str_arr=match.group().split()
            if(str_arr[0]=="Notes" or str_arr[0]=="Note"):
                str_arr.pop(0)
            break
    
#EXTRACT THE FINAL DATA AND CREATE A DICTIONARY IF POSSIBLE.

    index=-1
    financial_record={}
    
    # if str_arr is of the format [curr_year, prev_year], then financial data can be of the format:
    if(len(str_arr)==2):
        regex_1=re.compile(r'(.*)\s(\(?[\,\.0-9]+\)?|-)\s(\(?[\,\.0-9]+\)?|-)$') # 1) string Value1 Value2
        regex_2=re.compile(r'(\(?[\,0-9]+\)?|-)\s(\(?[\,0-9]+\)?|-)$') # 2) Value1 Value2
        regex_3=re.compile(r'(.*\s.*)$') # 3) string
        
    # if str_arr is of the format [curr_year], then financial data can be of the format:
    elif(len(str_arr)==1):
        regex_1=re.compile(r'(.*)\s(\(?[\,\.0-9]+\)?|-)$') # 1) string Value1
        regex_2=re.compile(r'(\(?[\,\.0-9]+\)?|-)$') # 2) Value1
        regex_3=re.compile(r'(.*\s.*)$') # 3) string
        
    if(len(str_arr)!=0):
        
        # find the index of "2019" if it is present in the file.
        if '2019' in str_arr:
            index=str_arr.index("2019")
            
        # extracting the label and its corresponding value (if it exists).
        for line in financial_data:
            label='nan'
            value='nan'
            
            if(regex_1.search(line)):
                match=regex_1.search(line)
                label=match.group(1).replace("Â£","&#163").replace("Â$","&#36").replace("Â€","&#8364")
                if(match.group(index+2)!='-' and index!=-1):
                    value=process_value(match.group(index+2))
                    
            elif(regex_2.search(line)):
                match=regex_2.search(line)
                if(match.group(index+1)!='-' and index!=-1):
                    value=process_value(match.group(index+1))
                    
            elif(regex_3.search(line)):
                match=regex_3.search(line)
                label=match.group(1).replace("Â£","&#163").replace("Â$","&#36").replace("Â€","&#8364")
                
            # appending the label and its value as key-value pair in a dictionary
            financial_record.update({str(label):str(value)})
                
    return json.dumps(financial_record)

In [13]:
def extract_balance_sheets_data(dir_path):
    file_names = os.listdir(dir_path)
    result=pd.DataFrame(columns=["Filename", "Extracted Values"])
    for file in file_names:
        if file in file_names[:4]:
            extracted_dic = extract_financial_data(os.path.join(dir_path, file))
        else:
            extracted_dic=json.dumps({})
        file_dic={
            "Filename":str(file.split(".")[0]),
            "Extracted Values":extracted_dic
        }
        result = result.append(file_dic, ignore_index="True")
    return result

In [14]:
#-----------------------------------------------------------------------------------------------------

In [15]:
dir_path = ".\HCL ML Challenge Dataset"
result=extract_balance_sheets_data(dir_path)

In [16]:
result.to_csv("result_1.csv",index=False)